In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("../input/all-the-data-set-needed/trainhacker1.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape# gets the number of rows and columns of a data set

In [ ]:
df.info()# gives all the information regarding the dataset

In [ ]:
df.isnull().sum()#check the null values present

In [ ]:

 df['Deal_value'] = df['Deal_value'].str.replace('$','')# replace $  in deal values

In [ ]:
df['Weighted_amount'] = df['Weighted_amount'].str.replace('$','')# replace $  in Weighted_amount 

In [ ]:
 df['Deal_value'] =df['Deal_value'].astype(float)#to convert to float datatype

In [ ]:
df['Deal_value']=df['Deal_value'].fillna(df['Deal_value'].mean())#then fill null values with mean

In [ ]:
df['Weighted_amount'] =df['Weighted_amount'].astype(float)
df['Weighted_amount']=df['Weighted_amount'].fillna(df['Weighted_amount'].mean())#similar for weighted_amount

In [ ]:
df.isnull().sum()

In [ ]:
sns.pairplot(df)# pairplot that gives the relation between different numerical varibles

In [ ]:
corre=df.corr()            #check for correlation :we can seee that deal value and weighted amount are highly correlated and hence any one can be removed if needed
sns.heatmap(corre)

In [ ]:
df.corr()

In [ ]:
df.drop(["Lead_name","Industry","Date_of_creation","Contact_no","POC_name","Designation","Lead_POC_email","Hiring_candidate_role","Lead_source","Last_lead_update","Internal_POC","Resource"], axis=1, inplace=True)

**Grouping with different categorical variables so that the mean sum etc of a particular group can be seen**

In [ ]:
m=df.groupby("Level_of_meeting")

In [ ]:
m["Success_probability"].mean()

In [ ]:
n=df.groupby("Geography")
n["Success_probability"].count()

In [ ]:
n["Success_probability"].mean()

In [ ]:
o=df.groupby("Fund_category")
o["Success_probability"].count()


In [ ]:
o["Success_probability"].mean()

In [ ]:
o=df.groupby("Pitch")
o["Success_probability"].mean()


In [ ]:
df=pd.concat([df,pd.get_dummies(df["Pitch"])],axis=1)
df=pd.concat([df,pd.get_dummies(df["Fund_category"])],axis=1)
df=pd.concat([df,pd.get_dummies(df["Level_of_meeting"])],axis=1)


In [ ]:
df.drop("Pitch",axis=1,inplace=True)
df.drop(["Fund_category","Level_of_meeting"],axis=1,inplace=True)

In [ ]:
df.drop(["Product_1","Category 4","Level 1"],axis=1,inplace=True)

In [ ]:
df

In [ ]:
cor=df.corr()
sns.heatmap(cor)

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, vmin=0, vmax=1)

In [ ]:
#from the data set we can see that locations in US have their stated in uppercase at the end 
df[['Place', 'State']] = df['Location'].str.split(',', 1, expand=True)#split location to place and state
df['State'].fillna('0', inplace = True)
df['Geography'].fillna('USA', inplace = True)
df.loc[df['State'] == '0', 'Geography'] = 'India'
df.drop(['Place', 'State','Location'], axis = 1, inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
l=[i for i in df.columns if df[i].dtype=="object" and i!="Deal_title"]
l

In [ ]:
from sklearn.preprocessing import LabelEncoder#label encoding
for i in l :
    
    df[i] = df[i].astype('category')
label_encoder = LabelEncoder()
for col in l:
    
    df[col] = label_encoder.fit_transform(df[col])

In [ ]:
X_train=df.drop(['Success_probability','Deal_title'],axis=1)

In [ ]:
y_train=df['Success_probability']

In [ ]:
!pip install xgboost

In [ ]:
import xgboost
regressor=xgboost.XGBRegressor()

In [ ]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [ ]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator= regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [ ]:
regressor=xgboost.XGBRegressor(base_score=0.75, booster='gblinear', colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.1, max_delta_step=None, max_depth=10,
       min_child_weight=4, monotone_constraints=None,
       n_estimators=500, n_jobs=12, num_parallel_tree=None,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=0, scale_pos_weight=1, subsample=None, tree_method=None,
       validate_parameters=1, verbosity=None)

In [ ]:
regressor.fit(X_train,y_train)

In [ ]:
df_test=pd.read_csv('../input/testdata-after-feature-enginerring/hackerearthcompetitiontest2.csv')


In [ ]:
y_pred=regressor.predict(df_test)
y_pred

In [ ]:
pred=pd.DataFrame(y_pred)
sub_df=pd.read_csv('../input/all-the-data-set-needed/testhacker1.csv')
datasets=pd.concat([sub_df['Deal_title'],pred],axis=1)
datasets.columns=['Deal_title','Success_probability']
datasets.to_csv('sample2.csv',index=False)

In [ ]:
from sklearn.metrics import accuracy_score
